In [32]:
import os
import fitz
from tqdm import tqdm

In [33]:
def check_rate_function(rate_1, rate_2, possible_rate):
        # 檢查兩種方法提取的評價是否相同
    if rate_1 == rate_2 and rate_1!= None:
        return rate_1
    elif rate_1 in possible_rate:
        return rate_1
    elif rate_2 in possible_rate:
        return rate_2
    else :
        return 'NULL'
            

In [34]:
zip_path = '玉山'
count_true = 0
possible_rate = ['買進(維持)', '逢低買進(維持)', '買進(初次)']
for directory in tqdm(os.listdir(zip_path)):
    directory_path = os.path.join(zip_path, directory)
    rate_1 = None
    rate_2 = None
    rate = None
    try:
        with fitz.open(directory_path) as doc:
            mat = fitz.Matrix(1, 1)
            page = doc.load_page(0)
            rect = page.rect
            page_check_source = doc.load_page(-1)
            text_check_source = page_check_source.get_text()
            if '玉山證券投資顧問' in text_check_source :
                # 檢查版本                
                clip_check_report = fitz.Rect(0, 0, rect.width, 100)
                text_check_report = page.get_text(clip=clip_check_report)
                if any(keyword in text_check_report for keyword in ['研 究 報 告']):
                    # 提取評價的第一種方法
                    try:
                        clip_old_report_1 = fitz.Rect(220, 80, rect.width, 180)
                        text_old_report_1 = page.get_text(clip=clip_old_report_1, sort=True).strip()
                        text_old_report_1 = text_old_report_1.split('TP')[0].strip()
                        rate_1 = text_old_report_1.split('\n')[-1].strip()
                    except:
                        rate_1 = None
                        # 提取評價的第二種方法
                    try:
                        clip_old_report_2 = fitz.Rect(230, 120, 430, 160)
                        rate_2 = page.get_text(clip=clip_old_report_2).strip()
                    except:
                        rate_2 = None
                # else :
                #     print(f'{text_check_report}, {directory_path} NULL')
                # if rate_1 == rate_2 and rate_1 not in possible_rate:
                #         possible_rate.append(rate_1)
                rate = check_rate_function(rate_1, rate_2, possible_rate)
            else:
                # False
                print(f'{directory_path} is not posted by 玉山投顧')
    except:
        print(directory_path, 'fail')
    if rate == 'NULL' :
        print('case1', directory_path, rate,
                f'rate_1={rate_1}, rate_2={rate_2}')
    elif rate == 'wrong':
        count_true += 1
        # print(directory_path, rate,
        #       f'text_check_new_report={text_check_new_report}, text_check_old_fast_report={text_check_old_fast_report}')
    elif rate != None :
        count_true += 1
        # print(directory_path, rate,
        #         f'rate_1={rate_1}, rate_2={rate_2}')
    else :
        print('case2', directory_path, rate,
                f'rate_1={rate_1}, rate_2={rate_2}')
# 計算成功和失敗的數量並打印
count_false = len(os.listdir(zip_path)) - count_true
success_rate = round((count_true/len(os.listdir(zip_path))), 3)*100
print(f'共{len(os.listdir(zip_path))}件, 成功{count_true}件, 失敗{count_false}件, 成功率:{success_rate}%')

100%|██████████| 5/5 [00:00<00:00, 33.57it/s]

共5件, 成功5件, 失敗0件, 成功率:100.0%


In [35]:
import fitz
doc = fitz.open(r'玉山\2014_中鴻_玉山_優於大盤.pdf')
mat = fitz.Matrix(1, 1)
page = doc.load_page(0)  # loads page number 'pno' of the document (0-based)
rect = page.rect
pix = page.get_pixmap(matrix=mat, alpha=False)
pix.save("page-%i.png" % page.number)

In [36]:
# 檢查是否由第一金投顧出版
page_check_source = doc.load_page(-1)
text_check_source = page_check_source.get_text()
if '玉山證券投資顧問' in text_check_source :
    print('True')

True


In [37]:
# 檢查版本
clip_check_report = fitz.Rect(0, 0, rect.width, 100)
pix = page.get_pixmap(matrix=mat, alpha=False, clip=clip_check_report)
pix.save("page-clip_check_report.png")
text_check_report = page.get_text(clip=clip_check_report)
if '研 究 報 告' in text_check_report :
    print('研 究 報 告')
else :
    print('NULL')
# text_check_report

研 究 報 告


In [38]:
# 報告為新版個股報告 提取評價的第一種方法
try:
    clip_old_report_1 = fitz.Rect(220, 80, rect.width, 180)
    text_old_report_1 = page.get_text(clip=clip_old_report_1, sort=True).strip()
    pix = page.get_pixmap(matrix=mat, alpha=False, clip=clip_old_report_1)
    pix.save("page-clip_new_report_1.png")
    text_old_report_1 = text_old_report_1.split('TP')[0].strip()
    rate_1 = text_old_report_1.split('\n')[-1].strip()
except:
    rate_1 = None
rate_1

'買進(維持)'

In [39]:
# 報告為新版個股報告 提取評價的第二種方法
try:
    clip_old_report_2 = fitz.Rect(230, 120, 430, 160)
    rate_2 = page.get_text(clip=clip_old_report_2).strip()
    pix = page.get_pixmap(matrix=mat, alpha=False, clip=clip_old_report_2)
    pix.save("page-clip_new_report_2.png")
except:
    rate_2 = None
rate_2

'買進(維持)'